<a href="https://colab.research.google.com/github/DrAlexSanz/Pix2Pix-DOTCSV/blob/master/Pix2Pix_DOTCSV.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This notebook is just for me, an implementation of the pix2pix algorithm following DotCSV's tutorial. It is very useful to get to know Tensorflow 2.0 and then I will use it for different things.

Video: [Video dotCSV](https://youtu.be/YsrMGcgfETY)

The tutorial, as he explains at the beginning, is based on this code:

[Tutorial Pix 2 Pix tensorflow 2.0](https://www.tensorflow.org/tutorials/generative/pix2pix)

In [1]:
%tensorflow_version 2.x

TensorFlow 2.x selected.


In [0]:
import tensorflow as tf
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import os, shutil
import glob
from PIL import Image, ImageOps, ImageFilter

# Define paths

# Root

PATH = "/content/drive/My Drive/Pix2PixFlowers"

# Input path

INPATH = PATH = "/content/drive/My Drive/Pix2PixFlowers/InputFlowers"

# Sketch path

SKEPATH = "/content/drive/My Drive/Pix2PixFlowers/SketchFlowers"

# Output path

OUTPATH = PATH = "/content/drive/My Drive/Pix2PixFlowers/OutputFlowers"

# Checkpoints directory

CKPATH = PATH = "/content/drive/My Drive/Pix2PixFlowers/Checkpoints"

Activate drive to load the data

In [9]:
from google.colab import drive
drive.mount('/content/drive', force_remount = True)

Mounted at /content/drive


In [10]:
a = INPATH + "/image_00001.jpg"
a = mpimg.imread(a)
plt.imshow(a)
plt.axis("off")

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py", line 2882, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-10-2bceda31f242>", line 2, in <module>
    a = mpimg.imread(a)
  File "/usr/local/lib/python3.6/dist-packages/matplotlib/image.py", line 1351, in imread
    with Image.open(fname) as image:
  File "/usr/local/lib/python3.6/dist-packages/PIL/Image.py", line 2530, in open
    fp = builtins.open(filename, "rb")
FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/My Drive/Pix2PixFlowers/InputFlowers/image_00001.jpg'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py", line 1823, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'FileNotFoundError' object has no attribute '_render_traceback_'

During handli

FileNotFoundError: ignored

In [0]:
def make_sketches(path_in, path_sketch):
    
    """
    This takes one of the real images and creates the "blurred" or modified images
    path in is where the input images are, path sketch is the place where the modified ones will be stored.
    """
    
    os.chdir(path_in)
    file_list = glob.glob("*")
    file_list.sort()
#   os.chdir(path_sketch)

    for file in file_list:
        a = Image.open(file)
        a = a.filter(ImageFilter.GaussianBlur(radius = 2))
#         a = a.filter(ImageFilter.Blur(radius = 4))
        a = ImageOps.posterize(a, 4)
        b = np.asarray(a).copy()
        b[:, :, 1] = 0
        im = Image.fromarray(np.uint8(b))
        os.chdir(path_sketch)
        im.save(file)
        os.chdir(path_in)
#         print(b.shape)
        
        
    
    return None

In [12]:
make_sketches(INPATH, SKEPATH)

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py", line 2882, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-12-85e69539ee5d>", line 1, in <module>
    make_sketches(INPATH, SKEPATH)
  File "<ipython-input-11-e28a5f06a12a>", line 8, in make_sketches
    os.chdir(path_in)
FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/My Drive/Pix2PixFlowers/InputFlowers'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py", line 1823, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'FileNotFoundError' object has no attribute '_render_traceback_'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/IPython/core/ultratb.py",

FileNotFoundError: ignored